In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-dmfvz074
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-dmfvz074
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=a83d76f26ce10a44d9a3c63584bc896cf0a473dd228e6e386a5af274024bb21a
  Stored in directory: /tmp/pip-ephem-wheel-cache-wjjeewwd/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
import clip
from PIL import Image

# Check if GPU available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device used: {device}")

# load CLIP model
model, preprocess = clip.load("ViT-B/32", device=device)

# check if the correct model is loaded
print(model)

Device used: cuda


100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 66.9MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

dataset_path = "/content/drive/MyDrive/AID_dataset/AID/"
print("Checking contents of the dataset folder:")
print(os.listdir(dataset_path))

Checking contents of the dataset folder:
['RailwayStation', 'SparseResidential', 'River', 'Viaduct', 'Square', 'Resort', 'Port', 'School', 'StorageTanks', 'Stadium', 'MediumResidential', 'Park', 'Meadow', 'Industrial', 'Playground', 'Mountain', 'Pond', 'Farmland', 'Parking', 'Forest', 'BareLand', 'BaseballField', 'DenseResidential', 'Desert', 'Commercial', 'Airport', 'Bridge', 'Beach', 'Center', 'Church']


In [ ]:
class_labels = {
    "Airport": "a satellite image of an airport",
    "BareLand": "a satellite image of bare land",
    "BaseballField": "a satellite image of baseball field",
    "Beach": "a satellite image of a beach",
    "Bridge": "a satellite image of a bridge",
    "Center": "a satellite image of a city center",
    "Church": "a satellite image of a church",
    "Commercial": "a satellite image of a commercial area",
    "DenseResidential": "a satellite image of a dense residential area",
    "Desert": "a satellite image of a desert",
    "Farmland": "a satellite image of farmland",
    "Forest": "a satellite image of a forest",
    "Industrial": "a satellite image of an industrial area",
    "Meadow": "a satellite image of a meadow",
    "MediumResidential": "a satellite image of a medium residential area",
    "Mountain": "a satellite image of a mountain",
    "Park": "a satellite image of a park",
    "Parking": "a satellite image of a parking lot",
    "Playground": "a satellite image of a playground",
    "Pond": "a satellite image of a pond",
    "Port": "a satellite image of a port",
    "RailwayStation": "a satellite image of a railway station",
    "Resort": "a satellite image of a resort",
    "River": "a satellite image of a river",
    "School": "a satellite image of a school",
    "SparseResidential": "a satellite image of a sparse residential area",
    "Square": "a satellite image of a square",
    "Stadium": "a satellite image of a stadium",
    "StorageTanks": "a satellite image of storage tanks",
    "Viaduct": "a satellite image of a viaduct",
}


In [ ]:
from PIL import Image
import os

# Function to preprocess a single image
def preprocess_image(image_path):
    image = Image.open(image_path)
    return preprocess(image).unsqueeze(0).to(device)

sample_image_path = os.path.join(dataset_path, 'DenseResidential', 'denseresidential_1.jpg')  # Adjust with an actual image path
image_input = preprocess_image(sample_image_path)
print("Preprocessed image shape:", image_input.shape)


Preprocessed image shape: torch.Size([1, 3, 224, 224])


In [ ]:
# Tokenize text labels for CLIP
text_inputs = torch.cat([clip.tokenize(f"This is {label}") for label in class_labels.values()]).to(device)


In [ ]:
with torch.no_grad():
    # Get image and text embeddings
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

    # Normalize features
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Compute similarity between image and text embeddings
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Find the predicted class with the highest similarity score
predicted_class_idx = similarity.argmax().item()
predicted_class = list(class_labels.keys())[predicted_class_idx]
print(f"Predicted class: {predicted_class}")


Predicted class: MediumResidential


In [ ]:
import random

def classify_images(dataset_path, class_labels, num_images=50):
    correct_predictions = 0
    total_images = 0

    true_labels = []
    predicted_labels = []

    for class_name in class_labels.keys():
        class_folder = os.path.join(dataset_path, class_name)
        image_files = os.listdir(class_folder)

        selected_images = random.sample(image_files, num_images)

        for image_file in selected_images:
            image_path = os.path.join(class_folder, image_file)
            image_input = preprocess_image(image_path)

            with torch.no_grad():
                image_features = model.encode_image(image_input)
                image_features /= image_features.norm(dim=-1, keepdim=True)
                similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

            predicted_class_idx = similarity.argmax().item()
            predicted_class = list(class_labels.keys())[predicted_class_idx]

            true_labels.append(class_name)        # True label from folder name
            predicted_labels.append(predicted_class)  # Predicted label from CLIP

            if predicted_class == class_name:
                correct_predictions += 1

            total_images += 1

    accuracy = correct_predictions / total_images
    return accuracy, true_labels, predicted_labels

accuracy, true_labels, predicted_labels = classify_images(dataset_path, class_labels, num_images=50)
print(f"Classification accuracy: {accuracy * 100:.2f}%")



Classification accuracy: 58.00%


In [ ]:
random_accuracy = 1 / len(class_labels)
print(f"Random guessing accuracy: {random_accuracy * 100:.2f}%")


Random guessing accuracy: 3.33%


In [ ]:
print(f"True labels sample: {true_labels[:5]}")
print(f"Predicted labels sample: {predicted_labels[:5]}")

True labels sample: ['Airport', 'Airport', 'Airport', 'Airport', 'Airport']
Predicted labels sample: ['Airport', 'Airport', 'StorageTanks', 'Airport', 'Airport']


In [ ]:
from sklearn.metrics import classification_report

# Assuming true_labels and predicted_labels are available
print(classification_report(true_labels, predicted_labels, target_names=list(class_labels.keys())))


                   precision    recall  f1-score   support

          Airport       0.91      0.86      0.89        50
         BareLand       0.32      0.34      0.33        50
    BaseballField       0.91      0.96      0.93        50
            Beach       0.96      0.96      0.96        50
           Bridge       0.83      0.80      0.82        50
           Center       0.00      0.00      0.00        50
           Church       0.29      0.86      0.43        50
       Commercial       0.25      0.14      0.18        50
 DenseResidential       0.87      0.40      0.55        50
           Desert       0.89      0.64      0.74        50
         Farmland       0.78      0.72      0.75        50
           Forest       0.89      0.78      0.83        50
       Industrial       0.65      0.44      0.52        50
           Meadow       0.55      0.24      0.33        50
MediumResidential       0.13      0.08      0.10        50
         Mountain       1.00      0.72      0.84       

In [ ]:
import os
from sklearn.model_selection import train_test_split

# List all image paths and their corresponding labels
image_paths = []
labels = []

for class_name in class_labels.keys():
    class_folder = os.path.join(dataset_path, class_name)
    image_files = os.listdir(class_folder)

    for image_file in image_files:
        image_paths.append(os.path.join(class_folder, image_file))
        labels.append(class_name)

# Split data into training and testing sets
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.8, random_state=42
)

print(f"Training set size: {len(train_image_paths)}")
print(f"Testing set size: {len(test_image_paths)}")



Training set size: 2000
Testing set size: 8000


In [ ]:
import torch.nn as nn

class LinearModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, input_dim)  # Input and output dimensions are the same

    def forward(self, x):
        return nn.functional.normalize(self.linear(x), dim=-1)  # Normalize the output to ensure magnitude is 1

In [ ]:
from PIL import Image
import clip

# Function to preprocess and encode image
def get_image_features(image_path):
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)
    return image_features

# Function to encode text labels
def get_text_features(labels):
    text_inputs = torch.cat([clip.tokenize(f"This is {label}") for label in labels]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features


In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.Adam(model_f.parameters(), lr=1e-3)

batch_size = 50  # Adjust based on available memory
num_epochs = 25  # Number of training epochs

# Training loop
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Clear previous gradients

    # Get image and text features for a batch of training data
    image_features_batch = torch.cat([get_image_features(path) for path in train_image_paths[:batch_size]]).to(device)
    text_features_batch = get_text_features(train_labels[:batch_size])
    image_features_batch = image_features_batch.to(torch.float32).to(device)
    text_features_batch = text_features_batch.to(torch.float32).to(device)

    # Forward pass: Transform image features
    outputs = model_f(image_features_batch)

    # Compute loss
    loss = criterion(outputs, text_features_batch)

    # Backpropagation
    loss.backward()

    # Update model parameters
    optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/25], Loss: 0.0003
Epoch [2/25], Loss: 0.0003
Epoch [3/25], Loss: 0.0002
Epoch [4/25], Loss: 0.0002
Epoch [5/25], Loss: 0.0002
Epoch [6/25], Loss: 0.0002
Epoch [7/25], Loss: 0.0001
Epoch [8/25], Loss: 0.0001
Epoch [9/25], Loss: 0.0001
Epoch [10/25], Loss: 0.0001
Epoch [11/25], Loss: 0.0001
Epoch [12/25], Loss: 0.0001
Epoch [13/25], Loss: 0.0001
Epoch [14/25], Loss: 0.0001
Epoch [15/25], Loss: 0.0001
Epoch [16/25], Loss: 0.0001
Epoch [17/25], Loss: 0.0001
Epoch [18/25], Loss: 0.0001
Epoch [19/25], Loss: 0.0000
Epoch [20/25], Loss: 0.0000
Epoch [21/25], Loss: 0.0000
Epoch [22/25], Loss: 0.0000
Epoch [23/25], Loss: 0.0000
Epoch [24/25], Loss: 0.0000
Epoch [25/25], Loss: 0.0000


In [ ]:
# Testing the model on test data
model_f.eval()
correct_predictions = 0
total_images = 0

# Get text features for all class labels (one time for all classes) and ensure they are float32
text_features_all = get_text_features(class_labels.keys()).to(device).to(torch.float32)

for i in range(0, len(test_image_paths), batch_size):
    batch_image_paths = test_image_paths[i:i+batch_size]
    batch_labels = test_labels[i:i+batch_size]

    # Get image features for the batch and ensure they are float32
    image_features_batch = torch.cat([get_image_features(path) for path in batch_image_paths]).to(device).to(torch.float32)

    with torch.no_grad():
        # Transform image features and ensure they are float32
        transformed_image_features = model_f(image_features_batch.to(torch.float32))

        # Ensure similarity is calculated between float32 tensors
        similarity = (100.0 * transformed_image_features @ text_features_all.T).to(torch.float32).softmax(dim=-1)

    # Check the shape of similarity to ensure it's correct
    print(f"Similarity shape: {similarity.shape}")  # Should be [batch_size, len(class_labels)]
    print(f"Number of class labels: {len(class_labels)}")

    # Calculate predicted classes
    predicted_class_indices = similarity.argmax(dim=1).tolist()

    # Check predicted indices for out of range values
    print(f"Predicted class indices: {predicted_class_indices}")
    print(f"Max index: {max(predicted_class_indices)}, Min index: {min(predicted_class_indices)}")

    # Map predicted indices to class labels
    predicted_classes = [list(class_labels.keys())[idx] for idx in predicted_class_indices]

    # Compare predictions with true labels
    for predicted_class, true_class in zip(predicted_classes, batch_labels):
        if predicted_class == true_class:
            correct_predictions += 1
        total_images += 1

# Calculate accuracy
accuracy = correct_predictions / total_images
print(f"Test accuracy after adaptation: {accuracy * 100:.2f}%")





Similarity shape: torch.Size([50, 30])
Number of class labels: 30
Predicted class indices: [19, 8, 7, 8, 25, 19, 1, 16, 19, 1, 8, 16, 16, 17, 12, 18, 1, 29, 21, 26, 29, 7, 25, 28, 15, 29, 6, 18, 19, 26, 21, 8, 8, 15, 25, 13, 17, 0, 12, 19, 17, 21, 25, 4, 28, 26, 15, 10, 5, 1]
Max index: 29, Min index: 0
Similarity shape: torch.Size([50, 30])
Number of class labels: 30
Predicted class indices: [11, 26, 15, 26, 9, 23, 9, 29, 8, 8, 3, 23, 18, 0, 4, 12, 8, 29, 18, 8, 16, 2, 10, 16, 9, 18, 3, 20, 25, 6, 27, 3, 16, 12, 17, 1, 10, 9, 27, 23, 3, 23, 28, 9, 9, 1, 5, 23, 25, 21]
Max index: 29, Min index: 0
Similarity shape: torch.Size([50, 30])
Number of class labels: 30
Predicted class indices: [2, 15, 15, 7, 20, 16, 27, 23, 18, 7, 17, 20, 12, 3, 26, 12, 23, 2, 7, 12, 21, 12, 23, 18, 13, 8, 10, 12, 18, 3, 19, 9, 1, 17, 18, 28, 0, 20, 20, 16, 1, 20, 19, 21, 23, 20, 16, 3, 4, 25]
Max index: 28, Min index: 0
Similarity shape: torch.Size([50, 30])
Number of class labels: 30
Predicted class indices: